In [1]:
import psycopg2 as pg2
import pandas as pd

con = pg2.connect(host='this_postgres',
                  user='postgres',
                  password='password',
                  database='postgres')
con.autocommit = True
cur = con.cursor()

def select(sql):
    return pd.read_sql(sql,con)

Создаётся прямое соединение с PostgreSQL через драйвер psycopg2. Параметр autocommit = True означает, что каждая команда автоматически фиксируется в базе данных. Объект cur (курсор) используется для выполнения SQL-команд.​

Функция выполняет SQL-запрос и возвращает результат в виде pandas DataFrame. Использует прямое соединение psycopg2 (con) вместо SQLAlchemy engine.​

In [2]:
sql = '''
SELECT pid , query, *
FROM pg_stat_activity pgsa
WHERE pid <> pg_backend_pid()
'''

`pg_stat_activity` — это системное представление PostgreSQL, которое показывает информацию о всех текущих соединениях и их активности. В таблице содержится одна строка для каждого серверного процесса (подключения).​

Ключевые колонки pg_stat_activity
`pid`: ID процесса backend-сервера PostgreSQL для конкретного соединения​

`datname`: Имя базы данных, к которой подключён клиент​

`usename`: Имя пользователя, владеющего соединением​

`query`: Текст SQL-запроса, который выполняется или был выполнен последним​

`state`: Текущее состояние соединения (active, idle, idle in transaction и т.д.)​

`query_start`: Время начала выполнения текущего запроса​

`client_addr`: IP-адрес клиента​

Функция `pg_backend_pid()`
`pg_backend_pid()` возвращает ID процесса текущего соединения. Условие WHERE pid <> pg_backend_pid() исключает из результата ваше собственное соединение, показывая только другие активные подключения.​

Результат
`DataFrame` df содержит информацию обо всех других соединениях: кто подключён, какие запросы выполняются, как долго они работают и в каком состоянии находятся.​

In [3]:
df = select(sql)

In [4]:
df

,pid,query,datid,datname,pid,usesysid,usename,application_name,client_addr,client_hostname,...,xact_start,query_start,state_change,wait_event_type,wait_event,state,backend_xid,backend_xmin,query,backend_type
0,75,,None,None,75,NaN,None,,None,None,...,None,None,None,Activity,AutoVacuumMain,None,None,None,,autovacuum launcher
1,76,,None,None,76,10.0,postgres,,None,None,...,None,None,None,Activity,LogicalLauncherMain,None,None,None,,background worker
2,46,,None,None,46,NaN,None,,None,None,...,None,None,None,Activity,BgWriterHibernate,None,None,None,,background writer
3,45,,None,None,45,NaN,None,,None,None,...,None,None,None,Activity,CheckpointerMain,None,None,None,,checkpointer
4,47,,None,None,47,NaN,None,,None,None,...,None,None,None,Activity,WalWriterMain,None,None,None,,walwriter


Kill all

In [5]:
sql = '''
SELECT 
    pg_terminate_backend(pid) 
FROM 
    pg_stat_activity 
WHERE 
    -- don't kill my own connection!
    pid <> pg_backend_pid()

'''

In [6]:
cur.execute(sql)